In [2]:
import flatten_lattice as fl
import torch
from bert_models import LinearLatticeBert, LinearPOSBert
from encoding_utils import *
import pickle
from mask_utils import *
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer, AutoConfig
from latmask_bert_models import LatticeBertModel
import json
import matplotlib.pyplot as plt

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [16]:
STOPS = -1

# get flattened graphs 
processedgraphs = fl.get_processed_graph_data(fl.frenbase, -1, STOPS)


# get exploded candidates in case needed
resarrs = [fl.get_cover_paths(p)[0] for p in processedgraphs]
#resarrs = []
#fl.get_allcands(fl.frenbase, -1, resarrs)



def filter_greedy(pgraphs):
    res = []
    for p in pgraphs:
        tmp = []
        for i in range(1, len(p)):
            tmp.append(p[i-1])
            if p[i]['pos']<=p[i-1]['pos']:
                break
            elif i==len(p)-1:
                tmp.append(p[i])
        res.append(tmp)
    return res

if STOPS==1:
    processedgraphs = filter_greedy(processedgraphs)
    
def clean_empty(rarrs, pgraphs):
    # clean up empty examples
    for r in range(0, len(resarrs)):
        try:
            if len(rarrs[r])==0:
                print(r)
                del rarrs[r]
                del pgraphs[r]
        except:
            break
    assert len(rarrs)==len(pgraphs)
print(len(resarrs))
print(len(processedgraphs))
bert_tok = fl.bert_tok
mbart_tok = fl.mbart_tok



101
101


In [18]:
"""
import pickle

with open('./torchsaved/pgraphsall.pkl', 'wb') as file:
    pickle.dump(processedgraphs, file)
with open('./torchsaved/resarrsall.pkl', 'wb') as file:
    pickle.dump(resarrs, file)
"""



"\nimport pickle\n\nwith open('./torchsaved/pgraphsall.pkl', 'wb') as file:\n    pickle.dump(processedgraphs, file)\nwith open('./torchsaved/resarrsall.pkl', 'wb') as file:\n    pickle.dump(resarrs, file)\n"

In [9]:
# sanity check to make sure things look ok
p = processedgraphs[89]
tlist = fl.get_toklist(p)
decstr = bert_tok.decode(tlist)
print(len(tlist))
#for node in p:
    #print(node['token_idx'], " ", node['pos'])
print(decstr)

167
Two days before the trial of President Mohamed Morsi was to begin, they went down to the street. s. the street. s, who has been deposed, they went ousted President Mohamed Morsi began, the two men went down to the street. s. they were out on went to on into the took, the two men went down to they were out took to the street. s went into the the outgoingi began, the, the two they were took went into out down to the deposedi opens, the two they have taken took went out was due to start, they took went open begin, the two they were went out to open, they took went begin, the two opened were went to on out into the street s began down, the outgoing deposed Mohamed Morsi, they went down President


In [10]:
# TODO more sanity checking on whether or not we're recording the right number of branches
def nexts_sanity(pgraph):
    toklist = []
    next_tok = pgraph[0]['token_idx']
    next_pos = 0
    for p in pgraph:
        if next_tok == p['token_idx'] and next_pos == p['pos']:
            tmp = p['nexts'][0].split()
            next_tok = int(tmp[0])
            next_pos = int(tmp[1])
            print(bert_tok.decode(p['token_idx']))

def get_attmasks(pgraphs):
    res = []
    i = 0
    for p in pgraphs:
        res.append(connect_mat(p))
        print(i)
        i+=1
    return torch.stack(res).to(device)

In [11]:
get_attmasks(processedgraphs[:1]).shape

0


torch.Size([1, 500, 500])

In [ ]:
assert len(processedgraphs)==101
attmasks = get_attmasks(processedgraphs)
# without lower triangular
torch.save(attmasks, './torchsaved/attmasksallv3.pt')

#torch.save(attmasks, './torchsaved/fixlatattmasks.pt')
# Change this when you run stuff
# TODO make a dictionary mapping from this to STOPS variable
#attmasks = torch.load('./torchsaved/attmasksall.pt')

In [25]:
attmasks = torch.tril(attmasks)

In [8]:
#attmasks = torch.load('./torchsaved/fixlatattmasks.pt')

In [55]:
# Model setup
class LinearPOSBertV1(nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        #self.bert = LatticeBertModel(AutoConfig.from_pretrained('bert-base-cased'))
        self.bert = AutoModel.from_pretrained('bert-base-cased')
        self.probe = nn.Linear(self.bert.config.hidden_size, num_labels)
        self.to(device)

    def parameters(self):
        return self.probe.parameters()
  
    def forward(self, sentences, pos_ids=None, attmasks=None):
        with torch.no_grad(): # no training of BERT parameters
            if pos_ids==None:
                word_rep, sentence_rep = self.bert(sentences, return_dict=False)
            else:
                
                word_rep, sentence_rep = self.bert(sentences, position_ids=pos_ids, encoder_attention_mask=attmasks, attention_mask=attmasks, return_dict=False)
        return self.probe(word_rep)
    
# credit to tutorial by https://pageperso.lis-lab.fr/benoit.favre/pstaln/09_embedding_evaluation.html for 
# input / pre-processing setup
with open('./a3distrib/lab_vocab.json') as json_file:
    labels = json.load(json_file)
posbmodel = LinearPOSBertV1(len(list(labels.keys())))
t = torch.load("./a3distrib/ckpt/bertonewayv1.pth")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
len(labels)

44

In [31]:
posbmodel.load_state_dict(t)
posbmodel.eval()
torch.cuda.empty_cache()
print(torch.cuda.memory_allocated("cuda:0"))

1069140992


In [32]:
del t
print(torch.cuda.memory_allocated("cuda:0"))

635572736


In [34]:
sents, posids = create_inputs(processedgraphs)
# sanity check roughly how many nodes current input params are producing

In [51]:
def avg_nodes(sentences):
    zcnts = [500-list(sent).count(0) for sent in sentences]
    return sum(zcnts)/len(zcnts), max(zcnts)


avg_nodes(sents)

(226.24752475247524, 462)

In [38]:
"""
# get maps from token -> labels
with open('./tmaps.pkl', 'rb') as handle:
    tmaps = pickle.load(handle)
    
tmaps = []
for i in range(0, 99):
    with open('./torchsaved/tmapssplit/tmaps0_'+str(i)+'.pkl', 'rb') as file:
        tmaps.append(pickle.load(file)[0])
"""
""

''

In [42]:
N_EX = 101
tmaps = []
for i in range(0, N_EX):
    with open('./torchsaved/tmapsmasked/tmaps_'+str(i)+'.pkl', 'rb') as file:
        tmaps.append(pickle.load(file)[0])

In [ ]:
print(torch.cuda.memory_allocated("cuda:0"))
torch.cuda.empty_cache()

#with torch.no_grad():
    #latposylabels, tmaps = get_biglabset(1)
latposylabels = tmap_pos_goldlabels(tmaps, sents)

In [44]:
len(tmaps[0].keys())

66

In [11]:
#torch.save(tmaps, "./torchsaved/tmapsnewdone.pt")
#torch.save(latposylabels, "./torchsaved/latposylabels.pt")
#torch.save(sents, "./torchsaved/sents.pt")
#torch.save(posids, "./torchsaved/posids.pt")
#torch.save(latposylabels, "./torchsaved/betterposlatposylabels.pt")

In [45]:
latposylabels.shape

torch.Size([101, 500, 44])

In [46]:
def inp_id_test(lenvals):
    tmp = list(range(0, 500))
    res = []
    for i in range(0, lenvals):
        res.append(torch.tensor(tmp))
    return torch.stack(res).to(device)


def mod_posids(pids):
    cop = pids
    for p in cop:
        for i in range(0, len(p)):
            if i>0 and p[i]==0:
                p[i] = i
    return cop

def fix_posids(pids):
    cop = pids
    for p in cop:
        for i in range(0, len(p)):
            p[i] = i
    return cop

def check_accuracy(setpred, setlabels):
    cor = 0
    tot = 0
    for i in range(0, len(setpred)):
        ex = setpred[i]
        for j in range(0, len(ex)):
            if sum(setlabels[i][j])==0:
                continue
            elif torch.argmax(setlabels[i][j])==0:
                continue
            tot+=1
            if torch.argmax(ex[j])==torch.argmax(setlabels[i][j]):
                cor+=1
    return cor/tot

def identify_errors(setpred, setlabels):
    cor = 0
    tot = 0
    res = []
    for i in range(0, len(setpred)):
        errtmp = []
        ex = setpred[i]
        for j in range(0, len(ex)):
            if sum(setlabels[i][j])==0:
                continue
            elif torch.argmax(setlabels[i][j])==0:
                continue
            tot+=1
            if torch.argmax(ex[j])==torch.argmax(setlabels[i][j]):
                cor+=1
            else:
                tmp = {}
                tmp['index'] = j
                tmp['predvec'] = ex[j]
                tmp['goldvec'] = setlabels[i][j]
                tmp['predlabel'] = torch.argmax(ex[j])
                tmp['godlabel'] = torch.argmax(setlabels[i][j])
                errtmp.append(tmp)
        res.append(errtmp)
                
    return res

def check_randacc(setpred, setlabels):
    cor = 0
    tot = 0
    for i in range(0, len(setpred)):
        ex = setpred[i]
        for j in range(0, len(ex)):
            if sum(setlabels[i][j])==0:
                continue
            tot+=1
            if torch.argmax(torch.rand(52).to(device))==torch.argmax(setlabels[i][j]):
                cor+=1
    return cor/tot

In [54]:
torch.argmax(latposylabels[0], dim=1)

tensor([ 1,  2,  3,  3, 14, 16, 22, 23,  3,  3,  2,  3,  3,  3,  3,  3, 12,  4,
         4, 12,  4,  1,  2,  5, 16, 22,  3,  5,  1, 25,  2, 34,  5,  1,  5,  1,
         4, 10,  3,  4, 12,  1,  5,  1,  3,  5, 16, 22,  2, 34,  5,  1,  5,  1,
         4, 10,  3,  1,  3,  4,  4,  4,  1,  5,  1,  2,  4, 12, 25,  3,  5,  1,
         1,  2,  5, 16, 22,  2,  3, 19, 34,  2,  5, 16, 22,  5,  1,  5, 12,  4,
         4,  4, 12, 25,  4,  4,  8, 15,  4, 12,  1,  5, 12,  1,  2,  5,  4,  1,
         2,  5, 16, 22,  3,  5,  1, 25,  2, 34,  5,  1,  5,  1,  4, 10,  3,  5,
        16, 22,  3,  5, 16, 22,  2, 22,  2,  3, 19, 22,  5,  2, 34,  1, 34, 34,
        22,  4, 12, 25,  3,  5, 16, 22,  2,  5,  1, 25, 22,  5, 16,  1, 22, 25,
         5,  2, 34,  5,  1,  2,  5,  5,  1,  5,  1, 19, 16, 22,  2, 34,  5, 22,
         2,  1, 22, 25,  2, 34,  5, 25, 19,  5, 16, 22,  5, 22,  2,  3, 19,  1,
         2, 34, 34,  5,  1,  2, 34,  3,  3,  3,  5,  1, 16, 22, 16, 22,  2,  5,
        16, 22,  5,  1, 22, 25,  5, 12, 

In [53]:
sents[0]

tensor([  101,  1109,  1646,  1697,  1108,  1106,  3531,  8612,  3460,  2110,
         1302,  8212,  2586, 15147,  1182,  5286,   117,  1379,   122,   117,
         1381,  1107,  1126,  3098,  1106,  5622,  1646,  5052,  1107,  2935,
         1103,  4997,  4003,  1104,  4289,  1107,  1421,  1201,   119,  1381,
          117,  1107,  3403,  1104,  1646,  5052,  1106,  2147,  1103,  4997,
         4003,  1104,  4289,  1107,  1421,  1201,   119,  1113,  5286,   122,
         1379,  1381,  1107,  3403,  1104,  1126,  1381,   117,  5788,  1646,
         5052,  1107,  1107,   170,  6875,  1106,  1494,  1103,  1244,  1311,
         4997,  1126,  3098,  1106,  5622,  5052,  1107,  3403,   117,   122,
         1379,  1381,   117,  5788,  1379,   122,   188,  1204,  1381,   117,
         1107,  3403,   117,  1107,  1126,  3098,  1381,  1107,  1126,  2661,
         1106,  5622,  1646,  5052,  1107,  2935,  1103,  4997,  4003,  1104,
         4289,  1107,  1421,  1201,   119,  3098,  1106,  1525, 

In [48]:
sents, posids = create_inputs(processedgraphs)
pred1 = posbmodel(sents, mod_posids(posids), attmasks)
sents, posids = create_inputs(processedgraphs)
pred2 = posbmodel(sents, fix_posids(posids), attmasks)
sents, posids = create_inputs(processedgraphs)
pred3 = posbmodel(sents, mod_posids(posids), None)
sents, posids = create_inputs(processedgraphs)
pred4 = posbmodel(sents, fix_posids(posids), None)

            
print(check_accuracy(pred1, latposylabels))
print(check_accuracy(pred2, latposylabels))
print(check_accuracy(pred3, latposylabels))
print(check_accuracy(pred4, latposylabels))

0.0
0.0
0.8202827946768061
0.7848344423320659


In [50]:
pred2

tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        ...,

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]

In [20]:
pred1.shape

torch.Size([99, 500, 44])

In [22]:
with open('./torchsaved/sentsdecent.pkl', 'wb') as file:
    pickle.dump(sents, file)
    
with open('./torchsaved/posidsdecent.pkl', 'wb') as file:
    pickle.dump(posids, file)
    
with open('./torchsaved/attmasksdecent.pkl', 'wb') as file:
    pickle.dump(attmasks, file)

In [25]:
with open('./torchsaved/predsdecent.pkl', 'wb') as file:
    pickle.dump(pred1, file)
    
with open('./torchsaved/labelsdecent.pkl', 'wb') as file:
    pickle.dump(latposylabels, file)

In [27]:
pred_errs  = identify_errors(pred1, latposylabels)

In [28]:
lblist = [l for l in labels.keys()]

In [40]:
tot = 0
for e in errpairs.keys():
    tot+=errpairs[e]
print(tot)

4926


In [ ]:
import collections
# number of errors per lattice 
e_len_distr = [len(e) for e in pred_errs]
def get_tag_pred_distr(p_errs):
    tpdistr = []
    tgdistr = []
    for perr in p_errs:
        for e in perr:
            tpdistr.append(float(e['predlabel']))
            tgdistr.append(float(e['godlabel']))
    return tpdistr, tgdistr
# error tag

def granular_pred_distr(p_errs):
    gpdistr = []
    for perr in p_errs:
        tmppred = []
        tmpgold = []
        for e in perr:
            tmppred.append(lblist[int(e['predlabel'])])
            tmpgold.append(lblist[int(e['godlabel'])])
        gpdistr.append(list(zip(tmppred, tmpgold)))
    return gpdistr

def understand_errpairs (epairdistr):
    frequency = collections.Counter(epairdistr)
    fdict = dict(frequency)

    d = dict(sorted(fdict.items(), key=lambda item: item[1]))
    #print(fdict)
    for k in d.keys():
        print(lblist[int(k[0])], " ", lblist[int(k[1])], ": ", d[k])
    return d
        
ep_distr, eg_distr = get_tag_pred_distr(pred_errs)
errpairs = understand_errpairs(zip(ep_distr, eg_distr))
granpd = granular_pred_distr(pred_errs)
for i in range(0, len(granpd)):
    print(i, " ", granpd[i])

In [47]:

latposylabels[0]

tensor([[ 15.2313,  -3.2251,  -6.9445,  ..., -31.4852, -19.2721, -20.7780],
        [ -3.4530,   3.3715,   9.0444,  ...,  -8.5703, -12.5707, -13.5763],
        [ -5.2397,   0.5385,   2.7538,  ..., -14.1365, -36.7951, -15.4723],
        ...,
        [ 14.5528,  -4.9714, -18.5249,  ..., -51.4184, -28.4842, -25.7899],
        [ 14.5528,  -4.9714, -18.5249,  ..., -51.4184, -28.4842, -25.7899],
        [ 14.5528,  -4.9714, -18.5249,  ..., -51.4184, -28.4842, -25.7899]],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [33]:
r1 = torch.rand((6, 500, 52))
r2 = torch.rand((6, 500, 52))
print(mse(r1, r2))
print(loss(r1, r2))
print(mse(latposylabels, pred2))

tensor(0.1663)
tensor(0.7329)
tensor(45.7936, device='cuda:0', grad_fn=<MseLossBackward0>)


In [40]:
# Code needed when changing set of gold labels
def prepare_dataset(resset):
    x = []
    y = []
    for res in resset:
        curinps = []
        for r in res:
            try:
                toktmp = torch.tensor(bert_tok(clean_expanded(r)).input_ids)
                #print(toktmp.shape)
                if float(toktmp.shape[0])<MAX_LEN:
                    toktmp = torch.cat([toktmp, torch.zeros(MAX_LEN-toktmp.shape[0])])
                else:
                    toktmp = toktmp[:MAX_LEN]
                curinps.append(toktmp)
            except:
                print("weird error happened") 
        print(len(curinps))
        curouts = []
        tinp = torch.stack(curinps).long().to(device)
        print(tinp.shape)
        y.append(posbmodel(tinp))
        x.append(tinp)
        del tinp
        
        #print("error somewhere")
    return x, y

def get_labset_partial(explodeds, startind, amt):
    #print(torch.cuda.memory_allocated("cuda:0"))
    dsetx, dsety = prepare_dataset(explodeds[startind:startind+amt])
    print(len(dsetx))
    #print(torch.cuda.memory_allocated("cuda:0"))
    assert len(dsetx)==amt
    latposylabels, tmaps = lattice_pos_goldlabels(dsetx, dsety, sents[startind:startind+amt])
    #print(torch.cuda.memory_allocated("cuda:0"))
    del dsetx, dsety
    
    torch.cuda.empty_cache()
    #print(torch.cuda.memory_allocated("cuda:0"))
    return latposylabels, tmaps

def get_biglabset(split):
    #result = None
    #tmaps = []
    for i in range(0, int(len(resarrs)/split)):
        print("SUBSET - ", i)
        r, tmap = get_labset_partial(resarrs, i*split, split)
        #tmaps = tmaps + tmap
        """
        if result == None:
            result = r
        else:
            result = torch.cat((result, r))
        """
        torch.cuda.empty_cache()
        #torch.save(result, './plabels/latreposlabels0.pt')
        file = open('./torchsaved/tmapsmasked/tmaps_'+str(i*split)+'.pkl', 'wb')
        # dump information to that file
        pickle.dump(tmap, file)

        # close the file
        file.close()
        del r
        del tmap
        torch.cuda.empty_cache()
        print(torch.cuda.memory_allocated("cuda:0"))

def debug_newinput(linp, explinps, latout, explouts):
    i = 0
    tot = 0
    while linp[i]>0:
        print()
        print(bert_tok.decode(linp[i]), " " ,bert_tok.decode(explinps[0][i]))
        print(explinps[0][i])
        lv, li = torch.topk(latout[i], 3)
        ev, ei = torch.topk(explouts[6][i], 3)
        print(li, " ", ei)
        #if l==e:
        #    tot+=1
        i+=1
    print(tot/i)

In [ ]:
# code to generate stuff for gold
get_biglabset(1)

In [14]:
import os

In [31]:
# START embedding distance stuff

from scipy.spatial.distance import cosine
from more_itertools import locate

def ind_cos_dist (i1, i2, embed):
    return 1-cosine(embed[i1].cpu(), embed[i2].cpu())

def tok_cos_dist(embed1, embed2, inps1, inps2, tok):
    l1 = list(inps1)
    l2 = list(inps2)
    
    inds1 = find_indices(l1, tok)
    inds2 = find_indices(l2, tok)
    sims = []
    e1 = embed1.cpu()
    e2 = embed2.cpu()
    for i1 in inds1:
        for i2 in inds2:
            sims.append(1-cosine(e1[i1], e2[i2]))
    #assert l1[ind1]==l2[ind2]
    return sims

def tok_cos_notbase(embed1, embed2, inps1, inps2, tok):
    l1 = list(inps1)
    l2 = list(inps2)
    
    inds1 = find_indices(l1, tok)
    inds2 = find_indices(l2, tok)
    sims = []
    e1 = embed1.cpu()
    e2 = embed2.cpu()
    for i1 in range(0, len(inps1)):
        for i2 in range(0, len(inps2)):
            if inps1[i1]==inps2[i2]:
                continue
            sims.append(1-cosine(e1[i1], e2[i2]))
    return sims



In [ ]:
import matplotlib.pyplot as plt

def mean(lis):
    try:
        return sum(lis)/len(lis)
    except:
        return 0.8

def get_ex_distr(example, ex_cand, latembed=None, sents=None):
    if latembed == None:
        sents, posids = create_inputs(processedgraphs[example:example+1])
        out = latbert(sents, posids)
        latembed = out['last_hidden_state'][0]
    norm_inputs = torch.tensor([bert_tok(clean_expanded(resarrs[example][ex_cand])).input_ids])
    #norm_inputs
    with torch.no_grad():
        norm_out = basebert(input_ids=norm_inputs, return_dict=True, output_hidden_states=True)
    normembed = norm_out['last_hidden_state'][0]
    res = []
    for r in norm_inputs[0]:
        res.append(mean(tok_cos_dist(latembed, normembed, sents[0], norm_inputs[0], r)))
    
    return res

def get_ex_notbase(example, ex_cand, latembed=None, sents=None):
    if latembed == None:
        sents, posids = create_inputs(processedgraphs[example:example+1])
        out = latbert(sents, posids)
        latembed = out['last_hidden_state'][0]
    norm_inputs = torch.tensor([bert_tok(clean_expanded(resarrs[ex_cand][0])).input_ids])
    #norm_inputs
    with torch.no_grad():
        norm_out = basebert(input_ids=norm_inputs, return_dict=True, output_hidden_states=True)
    normembed = norm_out['last_hidden_state'][0]
    res = []
    res.append(mean(tok_cos_notbase(latembed, normembed, sents[0], norm_inputs[0], r)))
    
    return res

def get_diff_base(latembed, inp_toks):
    diff = []
    for i in range(len(inp_toks)):
        for j in range(len(inp_toks)):
            if inp_toks[i]==inp_toks[j]:
                diff.append(ind_cos_dist(i, j, latembed))
    return diff

def get_lat_distr(example):
    result = []
    tot = len(resarrs[example])
    if tot>100:
        tot = 100
    for i in range(tot):
        print(i)
        sents, posids = create_inputs(processedgraphs[example:example+1])
        out = latbert(sents, posids)
        latembed = out['last_hidden_state'][0]
        result.append(mean(get_ex_distr(example, i, latembed, sents)))
    print(len(resarrs[example]))
    return result

In [ ]:
sents, posids = create_inputs(processedgraphs[0:0+1])
out = latbert(sents, posids)
latembed = out['last_hidden_state'][0]
plt.hist(get_diff_base(latembed, sents[0]))

In [ ]:
res = []

get_ex_notbase(0, 5)


In [ ]:
plt.hist(res, bins = 2)

In [ ]:
res[9]

In [ ]:
print(norm_inputs)
print(sents)

In [ ]:
# Importing the relevant modules
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
import torch

device = "cuda:0"
# Loading the pre-trained BERT model
###################################
# Embeddings will be derived from
# the outputs of this model
model = BertModel.from_pretrained('bert-base-cased',
           output_hidden_states = True,).to(device)
# Setting up the tokenizer
###################################
# This is the same tokenizer that
# was used in the model to generate
# embeddings to ensure consistency
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
def bert_text_preparation(text, tokenizer):
    """Preparing the input for BERT
    
    Takes a string argument and performs
    pre-processing like adding special tokens,
    tokenization, tokens to ids, and tokens to
    segment ids. All tokens are mapped to seg-
    ment id = 1.
    
    Args:
        text (str): Text to be converted
        tokenizer (obj): Tokenizer object
            to convert text into BERT-re-
            adable tokens and ids
        
    Returns:
        list: List of BERT-readable tokens
        obj: Torch tensor with token ids
        obj: Torch tensor segment ids
    
    
    """
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens]).to(device)
    segments_tensors = torch.tensor([segments_ids]).to(device)

    return tokenized_text, tokens_tensor, segments_tensors

def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    """Get embeddings from an embedding model
    
    Args:
        tokens_tensor (obj): Torch tensor size [n_tokens]
            with token ids for each token in text
        segments_tensors (obj): Torch tensor size [n_tokens]
            with segment ids for each token in text
        model (obj): Embedding model to generate embeddings
            from token and segment ids
    
    Returns:
        list: List of list of floats of size
            [n_tokens, n_embedding_dimensions]
            containing embeddings for each token
    
    """
    
    # Gradient calculation id disabled
    # Model is in inference mode
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # Removing the first hidden state
        # The first state is the input state
        hidden_states = outputs[2][1:]

    # Getting embeddings from the final BERT layer
    token_embeddings = hidden_states[-1]
    # Collapsing the tensor into 1-dimension
    token_embeddings = torch.squeeze(token_embeddings, dim=0)
    # Converting torchtensors to lists
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

    return list_token_embeddings


In [ ]:
target_word_embeddings = []

for text in examples:
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
    
    # Find the position 'bank' in list of tokens
    word_index = tokenized_text.index('Obama')
    # Get the embedding for bank
    word_embedding = list_token_embeddings[word_index]

    target_word_embeddings.append(word_embedding)

In [ ]:
len(target_word_embeddings[0])

In [ ]:
bert_model = LinearLatticeBert(52)


In [ ]:
sents = torch.tensor(sents).to(device)
posids = torch.tensor(posids).to(device)

In [ ]:
out = bert_model(sents, posids)

In [ ]:

lhs = out['last_hidden_state'][0]

In [ ]:
list(sents[0])[14]

In [ ]:
ind_cos_dist(13, 14, lhs)

In [ ]:
inptest = bert_tok(clean_expanded(resarrs[0][1]))

In [ ]:
normal_bert = AutoModel.from_pretrained('bert-base-cased')

In [ ]:
normalout = normal_bert(torch.tensor([inptest.input_ids]),return_dict=True, output_hidden_states=True)
normalhs = normalout['last_hidden_state'][0]

In [ ]:
bert_tok(clean_expanded(resarrs[0][1]))

In [ ]:
tok_cos_dist(lhs,normalhs,  1109)